<a href="https://colab.research.google.com/github/JJungEEun/lotte_project/blob/lattehauma-patch-1-1/2%EC%B0%A8%EB%AA%A8%EB%8D%B8%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
df1=pd.read_csv('/content/drive/MyDrive/colab/lotte/cust4.csv')

In [3]:
df1['yes'] = df1['m_pd'].apply(lambda x: 1 if x=='M0032' else 0)

In [4]:
df1.head()

,Unnamed: 0,index,cust,rct_no_x,de_hr_1,hrvar,royal1,royal2,royal3,weekday,...,de_hr,ma_fem_dv,ages,zon_hlv,weat,rain_n,temp,l_pd,m_pd,yes
0,0,0,M430112881,A01000001113,a_4,9.266085,0.0,1.0,0.0,fri,...,10,여성,50대,Z17,sp,0.0,-4.2,L0013,M0058,0
1,1,1,M646853852,A01000002265,a_4,1.676490,1.0,0.0,0.0,fri,...,10,여성,40대,Z13,sp,0.0,-4.2,L0048,M0298,0
2,2,2,M430112881,A01000003148,a_4,9.266085,0.0,1.0,0.0,fri,...,10,여성,50대,Z17,sp,0.0,-4.2,L0013,M0058,0
3,3,3,M430112881,A01000003148,a_4,9.266085,0.0,1.0,0.0,fri,...,10,여성,50대,Z17,sp,0.0,-4.2,L0013,M0058,0
4,4,4,M430112881,A01000004946,a_4,9.266085,0.0,1.0,0.0,fri,...,10,여성,50대,Z17,sp,0.0,-4.2,L0013,M0058,0


In [5]:
df1['yes'].value_counts()

0    4240519
1     118727
Name: yes, dtype: int64

In [6]:
df1.columns

Index(['Unnamed: 0', 'index', 'cust', 'rct_no_x', 'de_hr_1', 'hrvar', 'royal1',
       'royal2', 'royal3', 'weekday', 'rct_no_y', 'de_hr', 'ma_fem_dv', 'ages',
       'zon_hlv', 'weat', 'rain_n', 'temp', 'l_pd', 'm_pd', 'yes'],
      dtype='object')

## 로지스틱 회귀모형

In [7]:
x=df1[['de_hr_1','ma_fem_dv','ages','zon_hlv','weat','rain_n','temp',
       'hrvar','royal1','royal2','royal3','weekday']]
y=df1['yes']

In [8]:
x.head()

,de_hr_1,ma_fem_dv,ages,zon_hlv,weat,rain_n,temp,hrvar,royal1,royal2,royal3,weekday
0,a_4,여성,50대,Z17,sp,0.0,-4.2,9.266085,0.0,1.0,0.0,fri
1,a_4,여성,40대,Z13,sp,0.0,-4.2,1.676490,1.0,0.0,0.0,fri
2,a_4,여성,50대,Z17,sp,0.0,-4.2,9.266085,0.0,1.0,0.0,fri
3,a_4,여성,50대,Z17,sp,0.0,-4.2,9.266085,0.0,1.0,0.0,fri
4,a_4,여성,50대,Z17,sp,0.0,-4.2,9.266085,0.0,1.0,0.0,fri


원핫인코딩..해야겠지?


In [9]:
from sklearn.preprocessing import LabelEncoder
classle=LabelEncoder()
x['de_hr_1']=classle.fit_transform(x['de_hr_1'].values)
x['weekday']=classle.fit_transform(x['weekday'].values)
x['ma_fem_dv']=classle.fit_transform(x['ma_fem_dv'].values)
x['ages']=classle.fit_transform(x['ages'].values)
x['zon_hlv']=classle.fit_transform(x['zon_hlv'].values)
x['weat']=classle.fit_transform(x['weat'].values)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test=train_test_split(x,y, test_size=0.8, random_state=1)

In [11]:
x_train.shape

(871849, 12)

In [12]:
from sklearn.linear_model import LogisticRegression
logit=LogisticRegression(C=1e2, multi_class='ovr', random_state=1,
                         max_iter=200)

In [13]:
cut=sum(y_train)/len(y_train)

In [15]:
from sklearn.metrics import accuracy_score

logit.fit(x_train,y_train)
y_train_pred = logit.predict_proba(x_train)[:,1] >= cut
y_test_pred = logit.predict_proba(x_test)[:,1] >= cut
print(accuracy_score(y_train, y_train_pred))
print(accuracy_score(y_test, y_test_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.5106067679150862
0.5113937415212549


In [66]:
x_1=df1[['de_hr_1','ma_fem_dv','ages','weat','rain_n']]
y_1=df1['yes']

In [67]:
from sklearn.preprocessing import LabelEncoder
classle=LabelEncoder()
#x_1['weekday']=classle.fit_transform(x_1['weekday'].values)
x_1['de_hr_1']=classle.fit_transform(x_1['ma_fem_dv'].values)
x_1['ma_fem_dv']=classle.fit_transform(x_1['ma_fem_dv'].values)
x_1['ages']=classle.fit_transform(x_1['ages'].values)
#x_1['zon_hlv']=classle.fit_transform(x_1['zon_hlv'].values)
x_1['weat']=classle.fit_transform(x_1['weat'].values)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [68]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test=train_test_split(x_1,y_1, test_size=0.8, random_state=1)

In [69]:
from sklearn.linear_model import LogisticRegression
logit=LogisticRegression(C=1e2, multi_class='ovr', random_state=1,
                         max_iter=200)

In [70]:
cut=sum(y_train)/len(y_train)

In [71]:
from sklearn.metrics import accuracy_score

logit.fit(x_train,y_train)
y_train_pred = logit.predict_proba(x_train)[:,1] >= cut
y_test_pred = logit.predict_proba(x_test)[:,1] >= cut
print(accuracy_score(y_train, y_train_pred))
print(accuracy_score(y_test, y_test_pred))

0.7033373898461774
0.7039402167289815


In [73]:
df2=pd.read_csv('/content/drive/MyDrive/colab/lotte/new_df.csv')

In [75]:
df2.head()
df2[df2['m_pd']=='M0032']

,Unnamed: 0,pd_c,pd_nm,clac_hlv_nm,clac_mcls_nm,l_pd,m_pd
167,167,PD0168,사과,과일,국산과일,L0007,M0032
168,168,PD0169,배,과일,국산과일,L0007,M0032
169,169,PD0170,감,과일,국산과일,L0007,M0032
170,170,PD0171,귤류,과일,국산과일,L0007,M0032
171,171,PD0172,유자/모과,과일,국산과일,L0007,M0032
172,172,PD0173,복숭아,과일,국산과일,L0007,M0032
173,173,PD0174,자두,과일,국산과일,L0007,M0032
174,174,PD0175,살구,과일,국산과일,L0007,M0032
175,175,PD0176,매실,과일,국산과일,L0007,M0032
176,176,PD0177,딸기,과일,국산과일,L0007,M0032
